In [34]:
import pandas as pd
import os
import numpy as np
from typing import List

pd.options.display.max_rows = 20
pd.options.display.max_columns = None

In [42]:
df = pd.read_csv(os.path.dirname(os.getcwd())+r"\data"+"\clean_dataset.csv")
df.shape

(11288, 19)

In [22]:
from pandas_profiling import ProfileReport
profile = df.profile_report()
profile

KeyboardInterrupt: 

In [ ]:
profile.to_file(os.getcwd()+"\\reports"+"\\df_before_cleaning.html")

In [43]:
SOURCE_PATH = os.path.dirname(os.getcwd())+r"\data"+"\clean_dataset.csv"
COLUMNS_NAN_REPLACE = ["terrace_area", "garden_area", "land_surface", "price"]
COLUMNS_OUTLIERS_IGNORE = ["terrace_area", "source", "garden_area", "postcode", "facades_number"]

filepath: str = SOURCE_PATH
columns_nan_to_replace: List[str] = COLUMNS_NAN_REPLACE
columns_outliers_ignore: List[str] = COLUMNS_OUTLIERS_IGNORE

df = pd.read_csv(filepath)
#deep copy to avoid warnings when indexing
df_cleaned = df.copy(deep=True)
df_cleaned.shape

(11288, 19)

replace nan with 0 when relevant

In [38]:
for column in COLUMNS_NAN_REPLACE:
    df_cleaned[column] = df_cleaned[column].transform(lambda x: np.nan if x == 0 else x)
df_cleaned.shape

(11288, 19)

remove if terrace/garden true but area is 0

In [39]:
print(f'Intial rows {len(df)}')
df_cleaned = df_cleaned[~(((df_cleaned.terrace==True) & (df_cleaned.terrace_area==0))|((df_cleaned.garden==True) & (df_cleaned.garden_area==0)))]
print(f'% kept rows {len(df_cleaned)/len(df)*100}')

Intial rows 11288
% kept rows 100.0


Calculating Tukey fences to deal with founded non-normal distributions
NOTE FOR GROUP: effective python way to check for how normal functions are? Maybe already in profiling report?

In [40]:
def describe_with_tukey_fences(dataframe: pd.DataFrame):
    dataframe_desc = dataframe.describe(percentiles=[0.95, 0.94, 0.75, 0.5, 0.25, 0.06, 0.05], include=np.number)
    dataframe_index = dataframe_desc.index.to_list()
    fence_tukey_min =[dataframe_desc.loc["25%",c] - 1.5 * (dataframe_desc.loc["75%",c] - dataframe_desc.loc["25%",c]) for c in  dataframe_desc.columns]
    fence_tukey_max =[dataframe_desc.loc["75%",c] + 1.5 * (dataframe_desc.loc["75%",c] - dataframe_desc.loc["25%",c]) for c in  dataframe_desc.columns]
    dataframe_desc = dataframe_desc.append(dict(zip(dataframe_desc.columns, fence_tukey_min)), ignore_index=True)
    dataframe_desc = dataframe_desc.append(dict(zip(dataframe_desc.columns, fence_tukey_max)), ignore_index=True)
    dataframe_index.append('fence_tukey_min')
    dataframe_index.append('fence_tukey_max')
    dataframe_desc.index = dataframe_index    
    return dataframe_desc
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

,source,postcode,price,rooms_number,area,terrace_area,garden_area,land_surface,facades_number
count,11288.000000,11288.000000,1.128800e+04,11288.000000,11288.000000,5012.000000,2498.000000,5.585000e+03,64.000000
mean,5.988661,4857.760454,4.927968e+05,3.199947,195.134036,28.397845,1039.672538,1.697102e+03,2.687500
std,0.150174,3181.093932,5.676955e+05,3.786751,219.598588,36.773554,4269.032348,1.942578e+04,0.852168
min,4.000000,1000.000000,2.500000e+04,0.000000,5.000000,1.000000,1.000000,1.000000e+00,1.000000
5%,6.000000,1040.000000,1.290000e+05,1.000000,60.000000,4.000000,20.000000,7.200000e+01,2.000000
6%,6.000000,1050.000000,1.375000e+05,1.000000,63.000000,4.000000,25.000000,8.000000e+01,2.000000
25%,6.000000,1420.000000,2.250000e+05,2.000000,97.000000,10.000000,80.000000,1.830000e+02,2.000000
50%,6.000000,4420.000000,3.290000e+05,3.000000,146.000000,18.000000,240.000000,4.350000e+02,2.500000
75%,6.000000,8300.000000,5.300000e+05,4.000000,227.000000,33.000000,724.000000,1.052000e+03,3.000000
94%,6.000000,9300.000000,1.295000e+06,6.000000,448.000000,80.000000,2512.060000,3.559680e+03,4.000000


In [44]:
print(f'Intial rows {len(df)}')
for column in df_desc:
    if column not in COLUMNS_OUTLIERS_IGNORE:
        fence_tukey_min = df_desc.loc["fence_tukey_min", column]
        fence_tukey_max = df_desc.loc["fence_tukey_max", column]
        p95 = df_desc.loc["95%", column]
        p94 = df_desc.loc["94%", column]
        p06 = df_desc.loc["6%", column]
        p05 = df_desc.loc["5%", column]
        #printing
        row_start = len(df_cleaned)
        if fence_tukey_max >= p95:
            df_cleaned= df_cleaned[df_cleaned[column]<fence_tukey_max]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} Tukey's max outliers in {column}")
        elif (fence_tukey_max < p95) and  (p95 != p94):
            df_cleaned= df_cleaned[df_cleaned[column]<p95]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} percentile max outliers in {column}")
        row_start = len(df_cleaned)
        if fence_tukey_min <= p05:
            df_cleaned= df_cleaned[df_cleaned[column]>fence_tukey_min]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} Tukey's min outliers in {column}")
        elif (fence_tukey_max < p05) and  (p05 != p06):
            df_cleaned= df_cleaned[df_cleaned[column]<p95]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} percentile min outliers in {column}")
print(f"Excluding {(1-len(df_cleaned)/len(df))*100}")

Intial rows 11288
Excluding 5.040751240255137 percentile max outliers in price
Excluding 3.0786453960257476 Tukey's max outliers in rooms_number
Excluding 1.8577341418808402 percentile max outliers in area
Excluding 1.402510788544531 percentile max outliers in land_surface
Excluding 10.940822111977322


In [ ]:
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

### Converting categorical variables and adding numerical variables
Categorical variables were analysed by assigning to them median price (target variable).

For postcode a limited number of values remained for some postcodes after removing outliers.

In [46]:
postcode_price = df_cleaned.groupby('postcode')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per postcode
print(postcode_price.sort_values(by="median", ascending=False).head(5))
print(postcode_price.sort_values(by="mean", ascending=False).head(5))

def get_postcode_price(x):
    postcode_price_median = postcode_price.loc[x,"median"]
    return postcode_price_median
df_cleaned["postcode_median_price"] = df_cleaned["postcode"].apply(get_postcode_price)
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

                min        max       mean     median  len
postcode                                                 
2330      1350000.0  1350000.0  1350000.0  1350000.0  1.0
2350      1150000.0  1150000.0  1150000.0  1150000.0  1.0
2970       389000.0  1250000.0   859600.0   925000.0  5.0
2360       845000.0   845000.0   845000.0   845000.0  1.0
2250       799000.0   799000.0   799000.0   799000.0  1.0
                min        max       mean     median  len
postcode                                                 
2330      1350000.0  1350000.0  1350000.0  1350000.0  1.0
2350      1150000.0  1150000.0  1150000.0  1150000.0  1.0
2970       389000.0  1250000.0   859600.0   925000.0  5.0
2360       845000.0   845000.0   845000.0   845000.0  1.0
2250       799000.0   799000.0   799000.0   799000.0  1.0


,source,postcode,price,rooms_number,area,terrace_area,garden_area,land_surface,facades_number,postcode_median_price
count,10053.000000,10053.000000,1.005300e+04,10053.000000,10053.000000,10053.000000,10053.000000,10053.000000,59.000000,1.005300e+04
mean,5.988262,4954.659206,3.713688e+05,2.764548,155.398488,11.101562,124.184721,292.947677,2.677966,3.352739e+05
std,0.152775,3187.617962,2.367212e+05,1.247029,84.261466,22.848054,860.183845,576.096255,0.839554,1.364711e+05
min,4.000000,1000.000000,2.500000e+04,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,6.500000e+04
5%,6.000000,1040.000000,1.250000e+05,1.000000,56.000000,0.000000,0.000000,0.000000,2.000000,1.675000e+05
6%,6.000000,1050.000000,1.300000e+05,1.000000,60.000000,0.000000,0.000000,0.000000,2.000000,1.700000e+05
25%,6.000000,1490.000000,2.150000e+05,2.000000,93.000000,0.000000,0.000000,0.000000,2.000000,2.375000e+05
50%,6.000000,4500.000000,2.990000e+05,3.000000,135.000000,0.000000,0.000000,0.000000,3.000000,2.984000e+05
75%,6.000000,8370.000000,4.500000e+05,3.000000,200.000000,15.000000,0.000000,312.000000,3.000000,3.990000e+05
94%,6.000000,9300.000000,8.250000e+05,5.000000,312.000000,44.000000,500.000000,1339.520000,4.000000,6.010000e+05


For building state more values were available, analysing median price

In [61]:
state_price = df_cleaned.groupby('building_state_agg')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(state_price.sort_values(by="median", ascending=False).head(5))
print(state_price.sort_values(by="mean", ascending=False).head(5))

def get_building_state_price(x):
    state_price_median = state_price.loc[x,"median"]
    return state_price_median

df_cleaned["building_state_median_price"] = df_cleaned["building_state_agg"].apply(get_building_state_price)
describe_with_tukey_fences(df_cleaned)

                        min        max           mean    median     len
building_state_agg                                                     
good                49500.0  1396000.0  389462.253529  320000.0  7581.0
renovated           70000.0  1395000.0  391143.325997  310000.0   727.0
to_renovate         25000.0  1350000.0  284525.072779  230000.0  1745.0
                        min        max           mean    median     len
building_state_agg                                                     
renovated           70000.0  1395000.0  391143.325997  310000.0   727.0
good                49500.0  1396000.0  389462.253529  320000.0  7581.0
to_renovate         25000.0  1350000.0  284525.072779  230000.0  1745.0


,source,postcode,price,rooms_number,area,terrace_area,garden_area,land_surface,facades_number,postcode_median_price,building_state_median_price,property_subtype_median_price,building_property_subtype_median_facades
count,10053.000000,10053.000000,1.005300e+04,10053.000000,10053.000000,10053.000000,10053.000000,10053.000000,59.000000,1.005300e+04,10053.000000,10053.000000,8823.000000
mean,5.988262,4954.659206,3.713688e+05,2.764548,155.398488,11.101562,124.184721,292.947677,2.677966,3.352739e+05,303654.630459,316466.875560,2.551173
std,0.152775,3187.617962,2.367212e+05,1.247029,84.261466,22.848054,860.183845,576.096255,0.839554,1.364711e+05,33855.173743,80490.296436,0.624506
min,4.000000,1000.000000,2.500000e+04,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,6.500000e+04,230000.000000,118000.000000,2.000000
5%,6.000000,1040.000000,1.250000e+05,1.000000,56.000000,0.000000,0.000000,0.000000,2.000000,1.675000e+05,230000.000000,282500.000000,2.000000
6%,6.000000,1050.000000,1.300000e+05,1.000000,60.000000,0.000000,0.000000,0.000000,2.000000,1.700000e+05,230000.000000,282500.000000,2.000000
25%,6.000000,1490.000000,2.150000e+05,2.000000,93.000000,0.000000,0.000000,0.000000,2.000000,2.375000e+05,320000.000000,282500.000000,2.000000
50%,6.000000,4500.000000,2.990000e+05,3.000000,135.000000,0.000000,0.000000,0.000000,3.000000,2.984000e+05,320000.000000,288000.000000,2.000000
75%,6.000000,8370.000000,4.500000e+05,3.000000,200.000000,15.000000,0.000000,312.000000,3.000000,3.990000e+05,320000.000000,310000.000000,3.000000
94%,6.000000,9300.000000,8.250000e+05,5.000000,312.000000,44.000000,500.000000,1339.520000,4.000000,6.010000e+05,320000.000000,540000.000000,4.000000


For subtypes more values were available too.

In [60]:
property_subtype_price = df_cleaned.groupby('property_subtype')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(property_subtype_price.sort_values(by="median", ascending=False).head(5))
print(property_subtype_price.sort_values(by="mean", ascending=False).head(5))

def get_property_subtype_price(x):
    subtype_price_median = property_subtype_price.loc[x,"median"]
    return subtype_price_median

df_cleaned["property_subtype_median_price"] = df_cleaned["property_subtype"].apply(get_property_subtype_price)

                           min        max           mean    median    len
property_subtype                                                         
EXCEPTIONAL_PROPERTY  150000.0  1395000.0  664456.451613  652500.0   62.0
VILLA                 165000.0  1395000.0  594689.735450  540000.0  567.0
CASTLE                535000.0   535000.0  535000.000000  535000.0    1.0
MANSION               158000.0  1395000.0  593085.442478  525000.0  113.0
PENTHOUSE              85000.0  1396000.0  570653.242424  495000.0  297.0
                           min        max           mean    median    len
property_subtype                                                         
EXCEPTIONAL_PROPERTY  150000.0  1395000.0  664456.451613  652500.0   62.0
MANOR_HOUSE           290000.0  1230000.0  609000.000000  435000.0    5.0
VILLA                 165000.0  1395000.0  594689.735450  540000.0  567.0
MANSION               158000.0  1395000.0  593085.442478  525000.0  113.0
PENTHOUSE              85000.0  139600

replacing categorical values of state with facades

In [59]:
df_cleaned[df_cleaned.property_subtype=='APARTMENT']

,source,postcode,house_is,property_subtype,price,rooms_number,area,equipped_kitchen_has,furnished,open_fire,terrace,terrace_area,garden,garden_area,land_surface,facades_number,swimming_pool_has,region,building_state_agg,postcode_median_price,building_state_median_price,property_subtype_median_price,building_property_subtype_median_facades
882,6,3500,False,APARTMENT,245000.0,3.0,100.0,False,False,False,True,0.0,False,0.0,0.0,NaN,False,F,good,247300.0,320000.0,282500.0,2.0
883,6,3540,False,APARTMENT,289000.0,3.0,154.0,False,False,False,True,0.0,False,0.0,0.0,NaN,False,F,good,299500.0,320000.0,282500.0,2.0
926,6,3540,False,APARTMENT,158000.0,2.0,80.0,False,False,False,False,0.0,False,0.0,0.0,NaN,False,F,good,299500.0,320000.0,282500.0,2.0
979,6,3800,False,APARTMENT,299000.0,0.0,294.0,False,False,False,False,0.0,False,0.0,0.0,NaN,False,F,good,299000.0,320000.0,282500.0,2.0
6033,6,1160,False,APARTMENT,380000.0,2.0,87.0,True,False,False,True,13.0,False,0.0,0.0,NaN,False,B,good,450000.0,320000.0,282500.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11280,4,6000,False,APARTMENT,99000.0,2.0,91.0,True,False,False,False,0.0,False,0.0,0.0,2.0,False,W,to_renovate,154500.0,230000.0,282500.0,2.0
11282,4,4000,False,APARTMENT,245000.0,2.0,103.0,False,False,False,True,5.0,False,0.0,0.0,2.0,False,W,good,225000.0,320000.0,282500.0,2.0
11283,4,8790,False,APARTMENT,250000.0,1.0,300.0,False,False,False,False,0.0,False,0.0,0.0,2.0,False,F,good,257000.0,320000.0,282500.0,2.0
11284,4,2018,False,APARTMENT,298000.0,1.0,71.0,True,False,False,True,12.0,False,0.0,0.0,1.0,False,F,good,443475.0,320000.0,282500.0,2.0


In [80]:
df_with_facades = df_cleaned[df_cleaned.facades_number > 0]

property_subtype_facades = df_with_facades.groupby('property_subtype')['facades_number'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(property_subtype_facades)
print(property_subtype_facades.sort_values(by="median", ascending=False).head(5))
print(property_subtype_facades.sort_values(by="mean", ascending=False).head(5))

def get_property_subtype_facades(x):
    try:
        property_subtype_facades_median = property_subtype_facades.loc[x,"median"]
    except:
        property_subtype_facades_median = None
        #print(x) #error with apartment_block
    return property_subtype_facades_median

df_cleaned["property_subtype_median_facades"] = df_cleaned["property_subtype"].apply(get_property_subtype_facades)

                      min  max      mean  median   len
property_subtype                                      
APARTMENT             1.0  4.0  2.380952     2.0  21.0
DUPLEX                3.0  3.0  3.000000     3.0   1.0
EXCEPTIONAL_PROPERTY  4.0  4.0  4.000000     4.0   1.0
FLAT_STUDIO           2.0  2.0  2.000000     2.0   1.0
HOUSE                 1.0  4.0  2.692308     3.0  26.0
LOFT                  3.0  3.0  3.000000     3.0   1.0
MANSION               2.0  2.0  2.000000     2.0   1.0
MIXED_USE_BUILDING    2.0  2.0  2.000000     2.0   1.0
TOWN_HOUSE            3.0  3.0  3.000000     3.0   2.0
VILLA                 4.0  4.0  4.000000     4.0   4.0
                      min  max      mean  median   len
property_subtype                                      
EXCEPTIONAL_PROPERTY  4.0  4.0  4.000000     4.0   1.0
VILLA                 4.0  4.0  4.000000     4.0   4.0
DUPLEX                3.0  3.0  3.000000     3.0   1.0
HOUSE                 1.0  4.0  2.692308     3.0  26.0
LOFT      

In [71]:
df_cleaned.shape

(10053, 24)

In [78]:
df_cleaned.to_csv("df_after_cleaning.csv")

In [77]:
df_cleaned.to_csv(os.getcwd()+"\\outputs"+"\\df_after_cleaning.csv")

In [79]:
df_cleaned

,source,postcode,house_is,property_subtype,price,rooms_number,area,equipped_kitchen_has,furnished,open_fire,terrace,terrace_area,garden,garden_area,land_surface,facades_number,swimming_pool_has,region,building_state_agg,postcode_median_price,building_state_median_price,property_subtype_median_price,building_property_subtype_median_facades,property_subtype_median_facades
0,6,4180,True,MIXED_USE_BUILDING,295000.0,3.0,242.0,True,False,False,True,36.0,True,1000.0,1403.0,NaN,False,W,good,229000.0,320000.0,310000.0,2.0,2.0
1,6,8730,True,VILLA,675000.0,4.0,349.0,True,False,False,False,0.0,True,977.0,1526.0,NaN,False,F,good,241000.0,320000.0,540000.0,4.0,4.0
2,6,4020,True,APARTMENT_BLOCK,250000.0,5.0,303.0,True,False,False,False,0.0,False,0.0,760.0,NaN,False,W,to_renovate,195000.0,230000.0,357500.0,NaN,NaN
3,6,1200,True,HOUSE,545000.0,4.0,235.0,True,True,False,False,0.0,False,0.0,63.0,NaN,False,B,renovated,445000.0,310000.0,288000.0,3.0,3.0
4,6,1190,True,MIXED_USE_BUILDING,500000.0,2.0,220.0,True,False,False,False,0.0,True,60.0,193.0,NaN,False,B,good,360000.0,320000.0,310000.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11282,4,4000,False,APARTMENT,245000.0,2.0,103.0,False,False,False,True,5.0,False,0.0,0.0,2.0,False,W,good,225000.0,320000.0,282500.0,2.0,2.0
11283,4,8790,False,APARTMENT,250000.0,1.0,300.0,False,False,False,False,0.0,False,0.0,0.0,2.0,False,F,good,257000.0,320000.0,282500.0,2.0,2.0
11284,4,2018,False,APARTMENT,298000.0,1.0,71.0,True,False,False,True,12.0,False,0.0,0.0,1.0,False,F,good,443475.0,320000.0,282500.0,2.0,2.0
11286,4,2000,False,FLAT_STUDIO,150000.0,1.0,40.0,True,False,False,False,0.0,False,0.0,0.0,2.0,False,F,to_renovate,497000.0,230000.0,149000.0,2.0,2.0
